In [70]:
from google.colab import files

uploaded = files.upload()


Saving BCH-USD.csv to BCH-USD (3).csv
Saving BTC-USD.csv to BTC-USD (3).csv
Saving ETH-USD.csv to ETH-USD (3).csv
Saving LTC-USD.csv to LTC-USD (3).csv


In [0]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [74]:
!pip install tensorboardcolab
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

tbc=TensorBoardColab()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://191e80f9c493.ngrok.io


In [0]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [76]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [0]:
import pandas as pd
from sklearn import preprocessing
from random import shuffle
from collections import deque
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Dropout, Dense, LSTM, BatchNormalization
import os
import io
from tensorboard.plugins.hparams import api as hp

In [0]:
SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT = "LTC-USD"
BATCH_SIZE = 64
EPOCHS = 20

In [0]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd','adamax']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [0]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [0]:
def preprocess_df(df):
    df = df.drop("future",1)
    
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
            
    df.dropna(inplace=True)
    
    sequential_data = []
    prev_mins = deque(maxlen = SEQ_LEN)

    for value in df.values:
        prev_mins.append([n for n in value[:-1]])
        if len(prev_mins) == SEQ_LEN:
            sequential_data.append([np.array(prev_mins), value[-1]])
        
    shuffle(sequential_data)
    buys = []
    sells = []
    
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
            
        elif target == 1:
            buys.append([seq, target])
    
    shuffle(buys)
    shuffle(sells)
    
    lower = min(len(buys), len(sells))
    
    buys = buys[:lower]
    sells = sells[:lower]
    
    sequential_data = buys + sells
    shuffle(sequential_data)
    
    X = []
    y = []
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
    return np.array(X), y

In [84]:
main_df = pd.DataFrame()
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    print(ratio)
    dataset = f"{ratio}.csv"
    df = pd.read_csv(io.BytesIO(uploaded[dataset]), names=["time","low","high","open","close","volume"])
    df.rename(columns = {"close": f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace = True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    if len(main_df)==0:
        main_df = df
    else:
        main_df = main_df.join(df)       


main_df.fillna(method = "ffill", inplace = True)
main_df.dropna(inplace = True)

main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))


print(main_df[[f"{RATIO_TO_PREDICT}_close", "future", "target"]].head(10))

times = sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]
print(last_5pct)

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

BTC-USD
LTC-USD
ETH-USD
BCH-USD
            LTC-USD_close     future  target
time                                        
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
1528969020      96.440002  96.400002       0
1528969080      96.470001  96.400002       0
1528969140      96.400002  96.400002       0
1528969200      96.400002  96.400002       0
1528969260      96.400002  96.449997       1
1534922100


In [85]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Don't buys: {train_y.count(0)}, buys:{train_y.count(1)}")
print(f"VALIDATION Don't buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 77922 validation: 3860
Don't buys: 38961, buys:38961
VALIDATION Don't buys: 1930, buys: 1930


In [0]:
def train_test_model(hparams):
  model = Sequential()
  model.add(LSTM(hparams[HP_NUM_UNITS], input_shape=(train_x.shape[1:]), return_sequences=True))
  model.add(Dropout(hparams[HP_DROPOUT]))
  model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

  model.add(LSTM(hparams[HP_NUM_UNITS], return_sequences=True))
  model.add(Dropout(hparams[HP_DROPOUT]))
  model.add(BatchNormalization())

  model.add(LSTM(hparams[HP_NUM_UNITS]))
  model.add(Dropout(hparams[HP_DROPOUT]))
  model.add(BatchNormalization())

  model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
  model.add(Dropout(hparams[HP_DROPOUT]))

  model.add(Dense(2, activation='softmax'))

  model.compile(optimizer=hparams[HP_OPTIMIZER],
                loss ='sparse_categorical_crossentropy',
                metrics = ['accuracy']
                )
  NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

  log_dir= os.path.join('logs',"{}".format(NAME))
  tensorboard = TensorBoard(log_dir=log_dir)

  model.fit(
      np.array(train_x), np.array(train_y),
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      callbacks = [tf.keras.callbacks.TensorBoard(log_dir),
      hp.KerasCallback(log_dir,hparams)
      ]
  )
  _, accuracy = model.evaluate(np.array(validation_x),np.array(validation_y))
  return accuracy


In [0]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [0]:
%tensorboard --logdir logs/hparam_tuning

In [0]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/20
1218/1218 [==============================] - 21s 17ms/step - loss: 0.7022 - accuracy: 0.5242
Epoch 2/20
1218/1218 [==============================] - 20s 16ms/step - loss: 0.6891 - accuracy: 0.5385
Epoch 3/20
1218/1218 [==============================] - 20s 16ms/step - loss: 0.6865 - accuracy: 0.5468
Epoch 4/20
1218/1218 [==============================] - 20s 16ms/step - loss: 0.6843 - accuracy: 0.5525
Epoch 5/20
1218/1218 [==============================] - 20s 17ms/step - loss: 0.6815 - accuracy: 0.5589
Epoch 6/20
1218/1218 [==============================] - 20s 16ms/step - loss: 0.6800 - accuracy: 0.5648
Epoch 7/20
1218/1218 [==============================] - 19s 16ms/step - loss: 0.6786 - accuracy: 0.5681
Epoch 8/20
1218/1218 [==============================] - 20s 16ms/step - loss: 0.6771 - accuracy: 0.5708
Epoch 9/20
1218/1218 [==============================] - 19s 16ms/step - loss: 0.6760 - 